In [1]:
import nltk
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET # ElementTree Library


In [2]:
tree = ET.parse('/Users/cesc/Downloads/DrugBank_database.xml')

In [44]:
entities = []
root = tree.getroot()
for drug in root.findall('{http://www.drugbank.ca}drug'):
    entities.append(drug.find('{http://www.drugbank.ca}name').text)
    
    # Look for product names with the drug
    for drug_prod in drug.findall('{http://www.drugbank.ca}products'):
        for drug_prod in drug_prod.iter('{http://www.drugbank.ca}product'):
            for drug_prod_name in drug_prod.iter('{http://www.drugbank.ca}name'):
                # Only append the product if is formed by one word
                if len(drug_prod_name.text.split(' ')) == 1:
                    entities.append(drug_prod_name.text)

# Retain only the unique entities
entities = set(entities)

In [46]:
# Write the 'entities' list into a file for later use
with(open('data/drugbank_entities_not_processed.txt', 'w')) as f:
    for i in entities:
        f.write(i + '\n')

In [54]:
# Read the file
with(open('data/drugbank_entities_not_processed.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()

len(drugbank_db)

19135

In [55]:
# Tokenize all the entities
drugbank_db_mod = []
for entity in drugbank_db:
    tokens = nltk.word_tokenize(entity)
    for token in tokens:
        drugbank_db_mod.append(token)
        
len(drugbank_db_mod)

50093

In [56]:
# Separate those entities with a dash
drugbank_db_mod2 = []
for token in drugbank_db_mod:
    if '-' in token:
        split_tokens = token.split('-')
        for token in split_tokens:
            drugbank_db_mod2.append(token)
    else:
        drugbank_db_mod2.append(token)

len(drugbank_db_mod2)

81527

In [57]:
# Set of common words that we don't want to include
common_words = set(stopwords.words('english'))

# Eliminate those entities that don't fullfill certain conditions
# Length > 3, is not completely numeric, no duplicates and is not a 'stopword'
drugbank_db_mod3 = []
for token in drugbank_db_mod2:
    if len(token) > 3 and not token.isnumeric() and token.lower() not in common_words:
        drugbank_db_mod3.append(token)
        
len(drugbank_db_mod3)

36553

In [62]:
drugbank_db_mod4 = []
for token in drugbank_db_mod3:
    for ch in token:
        if ch.isalpha():
            drugbank_db_mod4.append(token)
            break

len(drugbank_db_mod4)

drugbank_db_mod4 = set(drugbank_db_mod4)



In [70]:
drugbank_db_final = [x.lower() for x in drugbank_db_mod4]
drugbank_db_final = set(drugbank_db_final)


with(open('data/DrugBank_names_DB.txt', 'w')) as f:
    for entity in drugbank_db_final:
        f.write(entity + '\n')    

In [71]:
sentence = 'Hello my name is Ibuprofeno and I like to take Aspirins from now and then Fluimucil and almax'
tokenized_sentence = nltk.word_tokenize(sentence)

for token in tokenized_sentence:
    for drug in drugbank_db_final:
        if drug in token.lower():
            print(token, drug)

Hello hello
Ibuprofeno ibuprofen
Ibuprofeno feno
Ibuprofeno profen
Ibuprofeno profeno
like like
Aspirins aspirin
Aspirins aspi


In [72]:
for token in tokenized_sentence:
    if token.lower() in drugbank_db_final:
        print(token)

Hello
like
